## Этапы (простой) обработки текста

<img src="images/textm.png">


## Декодирование


**Def.**  
перевод последовательности байт в последовательность символов

* Распаковка  
*plain/.zip/.gz/...*
* Кодировка  
*ASCII/utf-8/Windows-1251/...*
* Формат  
*csv/xml/json/doc...*

Кроме того: что такое документ?



## Разбиение на токены
**Def.**  
разбиение последовательности символов на части (токены), возможно, исключая из рассмотрения некоторые символы  
Наивный подход: разделить строку пробелами и выкинуть знаки препинания  


*Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру.*  


**Проблемы:**  
* example@example.com, 127.0.0.1
* С++, C#
* York University vs New York University
* Зависимость от языка (“Lebensversicherungsgesellschaftsangestellter”, “l’amour”)
Альтернатива: n-граммы

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
from nltk.tokenize import RegexpTokenizer


s = "Трисия любила Нью-Йорк, поскольку любовь к Нью-Йорку могла положительно повлиять на ее карьеру."

tokenizer = RegexpTokenizer("\w+|[^\w\s]+")
for t in tokenizer.tokenize(s): 
    print(t)

Трисия
любила
Нью
-
Йорк
,
поскольку
любовь
к
Нью
-
Йорку
могла
положительно
повлиять
на
ее
карьеру
.


## Стоп-слова
**Def.**  
Наиболее частые слова в языке, не содержащие никакой информации о содержании текста



In [ ]:
from nltk.corpus import stopwords


print(" ".join(stopwords.words("russian")[1:20]))

Проблема: “To be or not to be"

## Нормализация
**Def.**  
Приведение токенов к единому виду для того, чтобы избавиться от поверхностной разницы в написании  

Подходы  
* сформулировать набор правил, по которым преобразуется токен  
Нью-Йорк → нью-йорк → ньюйорк → ньюиорк
* явно хранить связи между токенами (WordNet – Princeton)  
машина → автомобиль, Windows 6→ window

In [ ]:
s = "Нью-Йорк"
s1 = s.lower()
print(s1)

In [ ]:
import re
s2 = re.sub(r"\W", "", s1, flags=re.U)
print(s2)

In [ ]:
s3 = re.sub(r"й", u"и", s2, flags=re.U)
print(s3)

## Стемминг и Лемматизация
**Def.**  
Приведение грамматических форм слова и однокоренных слов к единой основе (lemma):
* Stemming – с помощью простых эвристических правил
  * Porter (Cambridge – 1980)
        5 этапов, на каждом применяется набор правил, таких как
            sses → ss (caresses → caress)
            ies → i (ponies → poni)

  * Lovins (1968)
  * Paice (1990)
  * другие
* Lemmatization – с использованием словарей и морфологического анализа


## Стемминг

In [ ]:
from nltk.stem.snowball import PorterStemmer
from nltk.stem.snowball import RussianStemmer


s = PorterStemmer()
print(s.stem("Tokenization"))
print(s.stem("stemming"))

r = RussianStemmer()
print(r.stem("Авиация"))
print(r.stem("национальный"))

**Наблюдение**  
для сложных языков лучше подходит лемматизация

## Лемматизация

In [ ]:
# !pip install pymorphy2       

In [ ]:
import pymorphy2


morph = pymorphy2.MorphAnalyzer()
print(morph.parse("думающему")[0].normal_form)

## Heaps' law
Эмпирическая закономерность в лингвистике, описывающая распределение числа уникальных слов в документе (или наборе документов) как функцию от его длины.

$$
M = k T^\beta, \;M \text{ -- размер словаря}, \; T \text{ -- количество слов в корпусе}
$$
$$
30 \leq k \leq 100, \; b \approx 0.5
$$

<img src="images/dim.png">
<img src="images/heaps.png">

## Представление документов
**Boolean Model.** Присутствие или отсутствие слова в документе  
**Bag of Words.** Порядок токенов не важен  

*Погода была ужасная, принцесса была прекрасная.
Или все было наоборот?*

Координаты
* Мультиномиальные: количество токенов в документе
* Числовые: взвешенное количество токенов в документе

## Zipf's law
Эмпирическая закономерность распределения частоты слов естественного языка

$t_1, \ldots, t_N$ - токены, отранжированные по убыванию частоты
   	
$f_1, \dots, f_N$ - соответствующие частоты

**Закон Ципфа**
	$$
	f_i = \frac{c}{i^k}
	$$	
	
	Что еще? Посещаемость сайтов, количество друзей, население городов...
<img src="images/zipf.png">


In [7]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

s = pd.Series([
     "Авторитет и дружба вода и огонь, вещи разнородные и враждебные; равенство условие дружбы.",
     "Нет силы более могучей, чем знание; человек, вооружённый знанием, непобедим.",
     "Истинная сила человека не в порывах, а в нерушимом спокойствии.",
     "Величайшее богатство народа его язык."
     ],
    dtype="string"
)

# Код преобразования
import string

# Создаем множество из символов пунктуации
exclude = set(string.punctuation)

# Заменяем символы пунктуации на пустые строки
new_s = s.str.replace(f'[{string.punctuation}]', '', regex=True)
new_s = new_s.str.lower()

new_s = new_s.str.rstrip()
new_s = new_s.str.lstrip()

new_s = new_s.str.split()

print(new_s)

0    [авторитет, и, дружба, вода, и, огонь, вещи, р...
1    [нет, силы, более, могучей, чем, знание, челов...
2    [истинная, сила, человека, не, в, порывах, а, ...
3           [величайшее, богатство, народа, его, язык]
dtype: object


In [9]:
documents = ["I like this movie, it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer()

# Создаем the Bag-of-Words модель
bag_of_words = count_vectorizer.fit_transform(documents)

# Отобразим Bag-of-Words модель как DataFrame
feature_names = count_vectorizer.get_feature_names_out()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

,awesome,funny,hate,it,like,love,movie,nice,one,this,was
0,0,1,0,1,1,0,1,0,0,1,0
1,0,0,1,0,0,0,1,0,0,1,0
2,1,0,0,1,1,0,0,0,0,1,1
3,0,0,0,1,0,1,0,1,1,0,0


In [12]:
from nltk.util import ngrams
text = "I like this movie, it's funny. I hate this movie. This was awesome! I like it. Nice one. I love it."
tokenized = text.split()
bigrams = ngrams(tokenized, 2)
bigrams

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
document = ["I like this movie, it's funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']
tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(document)
# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names_out()
pd.DataFrame(values.toarray(), columns = feature_names)

,awesome,funny,hate,it,like,love,movie,nice,one,this,was
0,0.000000,0.571848,0.000000,0.365003,0.450852,0.000000,0.450852,0.000000,0.000000,0.365003,0.000000
1,0.000000,0.000000,0.702035,0.000000,0.000000,0.000000,0.553492,0.000000,0.000000,0.448100,0.000000
2,0.539445,0.000000,0.000000,0.344321,0.425305,0.000000,0.000000,0.000000,0.000000,0.344321,0.539445
3,0.000000,0.000000,0.000000,0.345783,0.000000,0.541736,0.000000,0.541736,0.541736,0.000000,0.000000
